In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib3

In [ ]:
# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
base_url = "https://health.ucdavis.edu/"
visited = set()
data = []

In [3]:
# Scraping function
def scrape_page(url):
    if url in visited:
        return
    visited.add(url)
    try:
        response = requests.get(url, verify=False)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        text = ' '.join([p.get_text() for p in soup.find_all('p')])
        data.append({"url": url, "text": text})
        
        time.sleep(3)  # Delay for 3 seconds
        
        for link in soup.find_all('a', href=True):
            link_url = link['href']
            if link_url.startswith('/'):
                link_url = base_url + link_url
            if base_url in link_url and link_url not in visited:
                scrape_page(link_url)
    except requests.exceptions.SSLError as e:
        print(f"SSL error for {url}: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Request error for {url}: {e}")

In [ ]:
# Start scraping from the base URL
scrape_page(base_url)

In [ ]:
# Save the data to a CSV file
df = pd.DataFrame(data)
df.to_csv('ucdavis_health_details.csv', index=False)
